In [1]:
import os
import json
import boto3

In [2]:
#initialize session
session = boto3.Session(
    profile_name=os.environ.get("BWB_PROFILE_NAME")
) #sets the profile name to use for AWS credentials

bedrock = session.client(
    service_name='bedrock-runtime', #creates a Bedrock client
    region_name='us-east-1',
    endpoint_url=os.environ.get("BWB_ENDPOINT_URL")
) 


In [5]:
def get_inference_parameters(model): #return a default set of parameters based on the model's provider
    bedrock_model_provider = model.split('.')[0] #grab the model provider from the first part of the model id
    
    if (bedrock_model_provider == 'anthropic'): #Anthropic model
        return { #anthropic
            "max_tokens_to_sample": 512,
            "temperature": 0, 
            "top_k": 250, 
            "top_p": 1, 
            "stop_sequences": ["\n\nHuman:"] 
           }
    
    elif (bedrock_model_provider == 'ai21'): #AI21
        return { #AI21
            "maxTokens": 512, 
            "temperature": 0, 
            "topP": 0.5, 
            "stopSequences": [], 
            "countPenalty": {"scale": 0 }, 
            "presencePenalty": {"scale": 0 }, 
            "frequencyPenalty": {"scale": 0 } 
           }
    
    elif (bedrock_model_provider == 'cohere'): #COHERE
        return {
            "max_tokens": 512,
            "temperature": 0,
            "p": 0.01,
            "k": 0,
            "stop_sequences": [],
            "return_likelihoods": "NONE"
        }
    
    elif (bedrock_model_provider == 'meta'): #META
        return {
            "temperature": 0,
            "top_p": 0.9,
            "max_gen_len": 512
        }

    else: #Amazon
        #For the LangChain Bedrock implementation, these parameters will be added to the 
        #textGenerationConfig item that LangChain creates for us
        return { 
            "maxTokenCount": 512, 
            "stopSequences": [], 
            "temperature": 0, 
            "topP": 0.9 
        }

## Streaming responses

We use Bedrock's invoke_model_with_response_stream function to make the call to the streaming API endpoint.
As response chunks are returned, this code extracts the chunk's text from the returned JSON and passes it to the provided callback method.

In [4]:
#callback handler for streaming
def chunk_handler(chunk):
    print(chunk, end='')


In [8]:
def get_streaming_response(bedrock_model_id, prompt, streaming_callback):

    model_kwargs = get_inference_parameters(bedrock_model_id) #get the default parameters based on the selected model
    
    model_kwargs['prompt'] = prompt

    body = json.dumps(model_kwargs)

    response = bedrock.invoke_model_with_response_stream(modelId=bedrock_model_id, body=body) #invoke the streaming method
    stream = response.get('body')
    if stream:
        for event in stream: #process each event returned by the stream
            chunk = event.get('chunk')
            if chunk:
                chunk_json = json.loads(chunk.get('bytes').decode())
                streaming_callback(chunk_json["completion"]) #pass the latest chunk's text to the callback method


In [9]:
prompt = "\n\nHuman:Tell me a story about two puppies and two kittens who became best friends\n\nAssistant:"
                
get_streaming_response('anthropic.claude-v2', prompt, chunk_handler)

 Here is a story about two puppies and two kittens who became best friends:

Floppy and Scruffy were two energetic puppy brothers who loved to play all day long. They would chase each other around the yard, wrestle and tumble, and chew on toys together. One day, their family brought home two kittens named Mittens and Boots. At first, the puppies barked excitedly at the new kittens, which scared the little furballs. The kittens hissed at the puppies and tried to swat them away. 

After a few days, the puppies calmed down and tried to get the kittens to play. They brought over balls and squeaky toys as gifts, but the kittens just batted them away. Floppy and Scruffy were sad that the kittens didn't want to play. Their mom told them to give the kittens some time to get used to their new home. So the puppies played with each other while keeping an eye on the kittens from across the room.  

Eventually, curiosity got the best of Mittens and Boots. They slowly crept up to the puppies while t

### Langchain

In [17]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms.bedrock import Bedrock

In [25]:
def get_streaming_response(bedrock_model_id, prompt):

    model_kwargs = get_inference_parameters(bedrock_model_id) #get the default parameters based on the selected model
   
    llm = Bedrock(
        model_id=bedrock_model_id,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
        model_kwargs=model_kwargs
    )
    
    return llm.predict(prompt) #return a response to the prompt



In [26]:
prompt = "\n\nHuman:Tell me a story about two puppies and two kittens who became best friends\n\nAssistant:"
                
get_streaming_response('anthropic.claude-v2', prompt)

 Here is a story about two puppies and two kittens who became best friends:

Floppy and Scruffy were two energetic puppy brothers who loved to play all day long. They would chase each other around the yard, wrestle and tumble, and chew on sticks and bones. One day, their family brought home two kittens named Mittens and Boots. At first, the puppies barked excitedly at the new kittens, which scared the little furballs. The kittens hissed at the puppies and tried to swat them away with their paws. 

After a few days, the puppies and kittens got used to each other. Floppy brought over a ball to Mittens, who hesitantly batted it back to him. Scruffy curled up next to Boots for a nap in a sunny patch of grass. The kittens realized the puppies just wanted to be friends. Soon, the four pets were inseparable. They played tag in the backyard, cuddled up for naps together, and shared meals side by side. At night, the puppies and kittens would snuggle up in a big pile of furry friends. Floppy, Sc

' Here is a story about two puppies and two kittens who became best friends:\n\nFloppy and Scruffy were two energetic puppy brothers who loved to play all day long. They would chase each other around the yard, wrestle and tumble, and chew on sticks and bones. One day, their family brought home two kittens named Mittens and Boots. At first, the puppies barked excitedly at the new kittens, which scared the little furballs. The kittens hissed at the puppies and tried to swat them away with their paws. \n\nAfter a few days, the puppies and kittens got used to each other. Floppy brought over a ball to Mittens, who hesitantly batted it back to him. Scruffy curled up next to Boots for a nap in a sunny patch of grass. The kittens realized the puppies just wanted to be friends. Soon, the four pets were inseparable. They played tag in the backyard, cuddled up for naps together, and shared meals side by side. At night, the puppies and kittens would snuggle up in a big pile of furry friends. Flopp

### Custom callback

In [43]:
from langchain.callbacks.base import BaseCallbackHandler

class SimpleCallback(BaseCallbackHandler):

    def __init__(self):
        pass

    def on_llm_start(self, serialized, prompts, **kwargs):
        print(f"LLM Start triggered with prompt - {prompts}")

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(token, end='--')
    
    def on_llm_end(self, *args, **kwargs) -> None:
        print("\nI'm done!")

In [44]:
def get_streaming_response(bedrock_model_id, prompt):

    model_kwargs = get_inference_parameters(bedrock_model_id) #get the default parameters based on the selected model
   
    llm = Bedrock(
        model_id=bedrock_model_id,
        streaming=True,
        callbacks=[SimpleCallback()],
        model_kwargs=model_kwargs
    )
    
    return llm.predict(prompt) #return a response to the prompt



In [45]:
prompt = "\n\nHuman:Tell me a one-paragraph joke\n\nAssistant:"
                
get_streaming_response('anthropic.claude-v2', prompt)

LLM Start triggered with prompt - ['\n\nHuman:Tell me a one-paragraph joke\n\nAssistant:']
 Here--'s-- a-- one-- paragraph-- joke--:--

A man-- walked-- into-- a-- bar-- and-- ordered-- a-- beer--.-- As-- the-- bartender-- went-- to-- get-- it--,-- he-- noticed-- a-- gor--illa-- sitting-- at-- the-- end-- of-- the-- bar--.-- He-- didn--'t-- think-- much-- of-- it-- and-- brought-- the-- man-- his-- drink--.-- A-- little-- while-- later--,-- the-- man-- ordered-- another-- beer-- and-- the-- bartender-- noticed-- the-- gor--illa-- was-- still-- sitting-- there--.-- The-- man-- ordered-- another-- beer--,-- and-- another--,-- and-- another--.-- By-- this-- point--,-- the-- bartender-- was-- getting-- curious-- about-- the-- gor--illa-- just-- sitting-- there--.-- So-- he-- finally-- asked-- the-- man--,-- "--Hey--,-- what--'s-- with-- the-- gor--illa--?-- He--'s-- been-- sitting-- there-- this-- whole-- time--."-- The-- man-- replied--,-- "--Oh-- him--?-- That--'s-- my-- pet-- gor--illa-

' Here\'s a one paragraph joke:\n\nA man walked into a bar and ordered a beer. As the bartender went to get it, he noticed a gorilla sitting at the end of the bar. He didn\'t think much of it and brought the man his drink. A little while later, the man ordered another beer and the bartender noticed the gorilla was still sitting there. The man ordered another beer, and another, and another. By this point, the bartender was getting curious about the gorilla just sitting there. So he finally asked the man, "Hey, what\'s with the gorilla? He\'s been sitting there this whole time." The man replied, "Oh him? That\'s my pet gorilla. His name is Mickey. I\'ll prove it to you." He turned to the gorilla and said, "Mickey, go get my wallet and pay the nice man." The gorilla got up, walked over to the man\'s jacket, took out his wallet, pulled out the cash to cover the bill and some tip, handed it to the bartender, and then sat back down. The bartender just stood there stunned and said, "Wow, that